$$
\begin{array}{c}
\LARGE \textbf{Trabajo\ Final\ de\ Comunicaciones\ Digitales}
\end{array}
$$

$$
\begin{aligned}
&\textbf{Introducción}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Marco teórico}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{} \\
&\text{El auge del Internet de las Cosas (IoT) ha impulsado el desarrollo de tecnologías de comunicación de} \\
&\text{bajo consumo y largo alcance, conocidas como Low Power Wide Area Networks (LPWAN). Dentro de este} \\
&\text{marco, LoRa (Long Range) es el estándar que se ha consolidado como una tecnología ideal para la} \\
&\text{comunicación inalámbrica de largo alcance y bajo consumo energético.} \\[1em]
&\text{En este sentido, el artículo propuesto por la cátedra introduce una formulación matemática para la} \\
&\text{modulación LoRa, la cual puede describirse como una modulación chirp con desplazamiento de frecuencia} \\
&\text{(Frequency Shift Chirp Modulation, FSCM).Esta técnica se basa en la codificación de bits en símbolos} \\
&\text{ enteros y en la generaciónde señales chirp cuya frecuencia depende de dichos símbolos.} \\
&\text{Además, en esta formulación, la información está codificada en el desplazamiento inicial de frecuencia } \\
&\text{del chirp,y no depende de la pendiente ni en la duración de la señal.Este enfoque permite definir una base } \\
&\text{ortogonal de señales discretas, lo que favorece una demodulación eficiente mediante la Transformada Rápida de}\\
&\text{Fourier (FFT).}\\
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Desarrollo}
\end{aligned}
$$

$$
 \begin{aligned}
 &\text{Este proceso de codificación, tal como se describe matemáticamente en el artículo, puede descomponerse en tres} \\
 &\text{etapas fundamentales. La primera de ellas es el mapeo de bits a un símbolo decimal, y comienza con un vector} \\
 &\text{ $w(nT_s)$ compuesto por una cantidad fija de bits determinada por el parámetro Spreading Factor (SF).} \\
 &\text{Este factor representa la cantidad de  bits por símbolo y se define como: } \\[1em]
 
 &\hspace{8cm} SF = \log_2(M) \\[1em]
 &\text{Donde $M$ es la cantidad total de símbolos posibles (cardinalidad de la modulación). Por ejemplo,} \\
 &\text{si $SF = 5$, existen $2^5 = 32$ símbolos distintos.} \\
 &\text{El valor decimal resultante de este mapeo, denotado como $s(nT_s)$, se obtiene al sumar los valores ponderados} \\
 &\text{de los bits del vector $w(nT_s)$. Este número representa directamente la frecuencia inicial desde la cual se} \\
 &\text{genera la señal chirp para su transmisión.} \\[1em]
 &\textbf{Fórmula de codificación:} \\[1em]
 &\hspace{8cm} s(nT_s) = \sum_{h=0}^{SF-1} w_h(nT_s) \cdot 2^h \\[1em]
 &\text{donde:} \\
 &\hspace{1cm} ~ s(nT_s) \in \{0, ..., 2^{SF} - 1\} \text{ es el símbolo codificado} \\
 &\hspace{1cm} ~ w_h(nT_s) \in \{0,1\} \text{ son los bits del bloque actual, ordenados desde el LSB} \\
 &\hspace{1cm} ~ SF \text{ es el Spreading Factor (cantidad de bits por símbolo)} \\
 &\hspace{1cm} ~ h \text{ es el índice del bit dentro del bloque} \\[1em]
 &\textbf{Parámetros de transmisión:} \\[1em]
 &\text{Supongamos que el ancho de banda del canal utilizado es $B$, lo que implica que transmitimos una muestra cada:} \\
 &\hspace{8cm} T = \frac{1}{B} \\[1em]
 &\text{Un símbolo $s(nT_s)$ se transmite cada:} \\
 &\hspace{8cm} T_s = 2^{SF} \cdot T \\[1em]
 &\text{Cada símbolo representa un número entero entre $0$ y $2^{SF} - 1$. Esto no solo determina la frecuencia} \\
 &\text{con la que inicia el chirp, sino también cuán "extendida" estará la señal en el tiempo. A mayor $SF$,} \\
 &\text{mayor será la duración del símbolo y mayor la resistencia al ruido, a costa de menor tasa de transmisión.} \\[1em]
 &\text{Por otro lado, el decodificador es el componente encargado de inferir el mensaje original a partir de la señal   recibida.} \\
 &\text{En un canal aditivo de ruido blanco gaussiano (AWGN), la estrategia óptima es la de Máxima Verosimilitud (ML).} \\
 &\text{Esta consiste en proyectar la señal recibida sobre todas las señales base posibles y elegir aquella cuya proyección} \\
 &\text{maximice la expresión:} \\[1em]
 &\hspace{8cm} \max_{c} \left| \langle y, c \rangle \right| \\[1em]
 &\text{Donde $c$ es una señal candidata y $y$ es la señal observada. Gracias a la ortogonalidad de las señales en FSCM,} \\
 &\text{esta proyección puede implementarse eficientemente con una FFT.}
 \end{aligned}
 $$

$$
\begin{aligned}
&\textbf{Implementación en Python}
\end{aligned}
$$

In [ ]:
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits en bloques de SF y los convierte en símbolos enteros."""
    bit_array = np.array(bit_array).reshape(-1, SF)
    powers = 2 ** np.arange(SF)[::-1]
    symbols = bit_array.dot(powers)
    return symbols

In [ ]:
def symbols_to_bits(symbols, SF):
    """Convierte los símbolos de vuelta a su representación binaria."""
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    return bits.reshape(-1)

In [ ]:
def simulate_encoder_decoder(SF, total_bits):
    """Genera bits aleatorios, codifica, decodifica y calcula el BER."""
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"

    # Generar bits aleatorios con distribución uniforme
    tx_bits = np.random.randint(0, 2, total_bits)

    # Codificación
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Transmisión simulada perfecta (sin ruido)
    rx_symbols = tx_symbols.copy()  # En canal real, se podría agregar ruido o errores

    # Decodificación
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # Cálculo de BER
    bit_errors = np.sum(tx_bits != rx_bits)
    ber = bit_errors / total_bits

    # Resultados
    print("Bits transmitidos (primeros 64):  ", tx_bits[:64])
    print("Bits decodificados (primeros 64):", rx_bits[:64])
    print(f"\nTotal de bits transmitidos: {total_bits}")
    print(f"Errores totales: {bit_errors}")
    print(f"BER (Bit Error Rate): {ber:.6f}")

# Parámetros de simulación
SF = 7  # Spreading Factor
num_bits = SF * 1000  # Enviar 1000 símbolos

simulate_encoder_decoder(SF, num_bits)

## Decodificación: Detección Óptima de Señales FSCM en Canales AWGN 

Como la señales tienen la misma energía y se supone sincronización perfecta en tiempp y frecuencia, así como símbolos igualmente probables, el receptor óptimo para señales FSCM en un canal AWGN puede derivarse directamente de la teoría clásica de detección de señales. 

Entonces, la fundamentación teórica del receptor óptimo es: 

### 1. Modelo del canal AWGN:
$$r = s + w$$ 

donde: 
- r: señal recibida (discreta). 
- s: chirp transmitido. 
- w: ruido blanco gaussiano. 

### 2. Dectector óptimo (MAP/ML):


### La señal recibida: 
$$r(nT_s + kT) = c(nT_s + kT) + w(nT_s + kT)$$ 

donde: 
- $r(nT_s + kT)$ es la muestra de la señal recibida. 
- $c(nT_s + kT)$ es la señal transmitida. 
- $w(nT_s + kT)$ es ruido blanco gaussiano de media cero. 

El detector óptimo en un canal AWGN elige el símbolo $\hat{s}(nT_s)$ que maximiza la correlación entre la señal recibida r y cada posible chirp $c_q:$

 $$\hat{s} = \arg\max_q \left| \sum_k r[k] \cdot c_q^*[k] \right|^2$$

Las formas de onda chirp definidas para la modulación FSCM poseen la propiedad de ortoganalidad: 

$$\langle c_q, c_{q'} \rangle = 0 \quad \text{si } q \neq q' $$

Esta propiedad asegura que la proyección de la señal recibida sibre una forma de onda incorrecta será nula (idelamente), y máxima cuando la proyección se realiza sobre el chirp correspondiente al símbolo transmitido.
La ortogonalidad es clave para permitir la detección ediciente mediante transformada rápida de Fourier (FFT)

Implementacion eficiente: 
Para esto se multiplica por un down - chirp y se aplica una FFT. El índice del pico de la FFT da el símbolo recibido y la complejidad se reduce de $O(M^2)$ a $O(M log M),$ con $M = 2^SF$

### Proceso óptimo de demodulación 
El demulador óptimo consiste en proyectar la señal recibida sobre todas las posibles formas $c(nT_s + kT)$ y selecciona aquella con la máxima correlación: 
$$\hat{s}(nT_s) = \arg\max_q \left| \langle r(nT_s + kT), c(nT_s + kT)|_{s(nT_s)=q} \rangle \right|^2$$ 

donde: 
- $q = 0,1, ..., 2^SF - 1$

1. Implementación computacionalmente eficiente 
Evitamos calcular todas las proyecciones explícitamente y usar la FFT 
pasos: 

 a. Multiplicación por un down- chirp: 

$$d(nT_s + kT) = r(nT_s + kT) \cdot e^{-j2\pi \cdot \frac{k^2}{2^{SF}}}$$
Equivale a bajar la frecuencia de la señal para que el chirp se convierta en una señal constante (o pico).


down-chirp: es una señal en la que la frecuencia disminuye linealmente con el tiempo y se expresa como 

$$\text{down\_chirp}[k] = e^{-j2\pi \cdot \frac{k^2}{2^{SF}}}$$

donde: 
- k es el índice de tiempo discreto(muestra)
- SF es el Spreading Factor 
- El exponente cuadrático en k genera un cambio de frecuencia con el tiempo. 
- El signo negativo (-) indica que la frecuencia decrece 

b. Aplicación de la Transformada de Fourier Rapida (FFT) 
Se toma la  FFT del vector $ d(nT_s)$ . El índice del pico en la salida de la DFT indica el símbolo transmitido: 
 $$\hat{s}(nT_s) = \arg\max_p |DFT[d(nT_s)]_p|^2$$ 

donde: 
- El vector $ d(nT_s)$ es el resultado de la multiplicación punto a punto entre la señal recibida y el down-chirp (del paso anterior)


## Ecuación 2 y 3: Modulación Chirp 

**Ecuación 2 (forma general):**

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi[(s(nT_s)+k) \bmod 2^{SF}]\frac{kT}{B}} \quad (2)$$

donde:
- $c(nT_s + kT)$ es la señal chirp modulada en el tiempo $nT_s + kT$
- $\frac{1}{\sqrt{2^{SF}}}$ es el factor de normalización de energía (asegura energía unitaria para cada símbolo)
- $s(nT_s)$ es el símbolo codificado (de la Ecuación 1)
- $k$ es el índice de muestra dentro del símbolo ($k = 0, 1, ..., 2^{SF}-1$)
- $T$ es el período de muestreo
- $B$ es el ancho de banda del canal
- $\bmod 2^{SF}$ indica la operación módulo $2^{SF}$

### Características principales:

1. **Chirp complejo**: La señal tiene frecuencia linealmente creciente dentro de cada símbolo
2. **Frecuencia instantánea**: Depende de $(s(nT_s)+k) \bmod 2^{SF}$, que representa un corrimiento de frecuencia específico para cada símbolo
3. **Ortogonalidad**: Los diferentes símbolos generan chirps ortogonales entre sí, facilitando la decodificación
4. **Normalización**: El factor $\frac{1}{\sqrt{2^{SF}}}$ mantiene la energía constante por símbolo


  
**Ecuación 3 (forma simplifca:**)


Dado que $T = \frac{1}{B}$, se reemplaza por $T \cdot B = 1$, simplificando la ecuación:

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi \cdot \frac{[(s(nT_s)+k) \bmod 2^{SF}] \cdot k}{2^{SF}}} \quad (3)$$

**Pasos** 

1. Definiciíon del tiempo discreto:  

Teniendo en cuento lo definido más arriba: 
 
$$t = nT_s + kT = n \cdot 2^ SF \cdot T + kT$$

2. Codificar la frecuencia en el chirp: 

La modulacíon LoRa usa un chirp de frecuencia creciente, donde la frecuencia instantánea varía linealmente con k, y 
se desplaza según el valor del símbolo $$s(nT_s)$$ 
$$\frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}$$


3. Fase acumulada y señal compleja 

$$e^{j2\pi \cdot \frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}}$$

Representa una señal compleja cuya fase cambia con el tiempo (índice k).Esa fase varía en funcíon del símbolo transmitido s y de la muestra actual k. 
Como resultado, se genera una onda chirp (señal cuya frecuencia aumenta linealmente con k ). Esto se logra porque la fase del exponente crece de forma cuadrática con k, algo característico de un chirp lineal. 
El desplazamiento s hace que cada símbolo comience su chirp en una frecuencia distinta, lo que permite codificar la información.

4. Normalizacíon de energía 

El factor $\frac{1}{\sqrt{2^{SF}}}$ normaliza la señal para que tenga energía unitaria. 

$\text{Implementación en python}$

<text>La función bits_to_symbols: Agrupa un arreglo plano de bits en bloques de longitud igual al Spreading Factor (SF) y 

convierte cada bloque en un número decimal. Este paso simula lo que en el paper se describe como la generación del símbolo
 
𝑠(𝑛𝑇𝑠) a partir del vector de bits 𝑤(𝑛𝑇𝑠)

In [ ]:
import random
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits